In [1]:
import cv2
import numpy as np
import math

In [2]:
cap=cv2.VideoCapture(0)

In [3]:
def findMaxcontours(contours):
    max_i=0 # max alanın bulunduğu indis c
    max_area=0 # şimdilik 0
    for i in range(len(contours)):
        face_area=cv2.contourArea(contours[i])
        if face_area>max_area:
            max_area=face_area
            max_i=i
        # hata alma ihtimaline karşı try except
        try:
            c=contours[max_i]
        except:
            # hiçbirşey bulamazsak c= 0 dicez yani
            contours=[0]
            c=contours[0]
        return c
while 1:
    _,frame=cap.read()
    frame=cv2.flip(frame,1)
    
    # belli bir bölgede yüzü aramamız lazım o bölgeye aramayı götürmeden önce roi tanımlamamız lazım
    # yüz ile ilgili uygulama yapacağımızdan ortalarda arayacağız
    roi=frame[115:400,175:430] # önce y ekseni sonra x ekseni
    
    # şimdi nereye denk geldiğini görmek için dikdörtgen çizdirip teğit edeceğiz
    cv2.rectangle(frame,(175,115),(430,400),(0,0,255),0)# pencere etrafında kırmızı şerit oluşmaması için kalınlığını 0 giriyorum 
    # artık yüzüm işlenmeye hazır
    # yüzümü ayırdım şimdi renklerden de ayırmak için yüzümü yani roi yi hsv koduna çevireceğim
    hsv=cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)
    # şimdi hazır bulduğumuz yüz için olan hsv kodlarını işleyeceğiz
    lower_color=np.array([139,45,103],dtype=np.uint8) # videodakini yapınca olmadı ben de kendi değerlerimi buldum
    upper_color=np.array([180,255,255],dtype=np.uint8)
    mask=cv2.inRange(hsv,lower_color,upper_color)
    # mask da oluşan görüntüye curnel medyanbulur uygulayıp karıncalanmaları yok edeceğim
    kernal=np.ones((3,3),dtype=np.uint8)
    mask=cv2.dilate(mask,kernal,iterations=1)
    mask=cv2.medianBlur(mask,15)
    #şimdi görüntüler içerisinde counters arayacağız
    contours,_=cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    # yüzümüzün en uç noktalarını arayacağız eğer counters değer döndürmediyse ihtimalini de ele alıp if oluşturucaz
    if len(contours)>0:
        # burada da hata alma ihtimalimiz var o yüzden try except block oluşturuyoruz
        try:
            # yüzüm countur lar içindeki en büyük nesne olacağından en büyük nesneyi buldurucam
            c=findMaxcontours(contours) # max counter bulunan değer
            # şimdi uç noktaları atayalım
            extLeft = tuple(c[c[:, :, 0].argmin()][0]) # bu bana enleri veriyor
            extRight = tuple(c[c[:, :, 0].argmax()][0])
            extTop = tuple(c[c[:, :, 1].argmin()][0])
            extDown = tuple(c[c[:, :, 1].argmax()][0])
            
            # şimdi bulduğum yerlere çemberler koyacağım
            cv2.circle(roi,extLeft,5,(0,255,0),2)
            cv2.circle(roi,extRight,5,(0,255,0),2)
            cv2.circle(roi,extTop,5,(0,255,0),2)
            cv2.circle(roi,extDown,5,(0,255,0),2)
            
            # şimdi çemberleri birleştireceğim
            cv2.line(roi,extLeft,extTop,(255,0,0),2)
            cv2.line(roi,extTop,extRight,(255,0,0),2)
            cv2.line(roi,extRight,extDown,(255,0,0),2)
            cv2.line(roi,extDown,extLeft,(255,0,0),2)
            
            # şimdi yüzümdeki uç noktalar arasındaki açıları ve uzunlukları bulacağız
            # ilk önce uzunlukları buluyoruz
            ust_enSag_uzunlugu = math.sqrt((extRight[0]-extTop[0])**2+(extRight[1]-extTop[1])**2)
            enSag_alt_uzunlugu = math.sqrt((extDown[0]-extRight[0])**2+(extDown[1]-extRight[1])**2)
            ust_alt_uzunlugu = math.sqrt((extDown[0]-extTop[0])**2+(extDown[1]-extTop[1])**2)
            # şimdi uzunluklarını bildiğimizden cos teoremi sayesinde 2 kenar arasındaki açıyı bulacağız
            # /(2*enSag_alt_uzunlugu*ust_alt_uzunlugu) = 0 gelme ihtimali var o yüzden tyr except biloğuna alıyoruz
            try:
                angle_ustSag_altSAg_kenar= int(math.acos((ust_enSag_uzunlugu**2+enSag_alt_uzunlugu**2-ust_alt_uzunlugu**2)/(2*enSag_alt_uzunlugu*ust_alt_uzunlugu))*57)# angle=açı içerisi cos teoreminde kenarları bilip açıyı yanlız bıraktığında oluşan matematik işlemi ama *57 anlamadım
                cv2.putText(roi,str(angle_ustSag_altSAg_kenar),(extRight[0]-100+50,extRight[1]),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2,cv2.LINE_AA)# sondaki fontla alakalı güzel gözüksün diye
            except:
                cv2.putText(roi," ? ",(extRight[0]-100+50,extRight[1]),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2,cv2.LINE_AA)
        except:
            pass
    else:
        pass
    
    cv2.imshow("frame",frame)
    cv2.imshow("roi",roi)
    cv2.imshow("mask",mask)
    
    if cv2.waitKey(20)& 0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

# Neden *57 : 
Kodun içinde görülen *57 ifadesi, radyan cinsinden hesaplanan bir açı değerini dereceye çevirmek için kullanılır. Çünkü Python'da trigonometrik fonksiyonlar (math.acos() gibi) genellikle radyan cinsinden çalışır. Ancak açıların genellikle derece cinsinden ifade edilmesi tercih edilir. Bu nedenle, radyan cinsinden hesaplanan bir açıyı dereceye çevirmek için 57.29577951308232 (180/pi yaklaşımı) değeri kullanılır.